## 配置环境，生成.so文件

In [12]:
from tree_sitter import Language
Language.build_library(
  # so文件保存位置
  'build/my-languages.so',

  # vendor文件下git clone的仓库
  [
    'vendor/tree-sitter-java',
    'vendor/tree-sitter-python',
    'vendor/tree-sitter-cpp',
    'vendor/tree-sitter-c-sharp',
    'vendor/tree-sitter-javascript',
    'vendor/tree-sitter-go',
    'vendor/tree-sitter-php',
    'vendor/tree-sitter-typescript/tsx/',
    'vendor/tree-sitter-css',
    'vendor/tree-sitter-prolog',
    'vendor/tree-sitter-r',
    'vendor/tree-sitter-rust',
    'vendor/tree-sitter-c',
    'vendor/tree-sitter-cuda',
    'vendor/tree-sitter-perl',
    'vendor/tree-sitter-scala',
    'vendor/tree-sitter-matlab',
    'vendor/tree-sitter-pascal',
    'vendor/tree-sitter-ruby',
    'vendor/tree-sitter-lua',
    #'vendor/tree-sitter-lean',
    'vendor/tree-sitter-dart',
    'vendor/tree-sitter-kotlin',
    'vendor/tree-sitter-html',
    'vendor/tree-sitter-elisp',
    'vendor/tree-sitter-latex',
    #'vendor/tree-sitter-powershell',
    'vendor/tree-sitter-sqlite',
    'vendor/tree-sitter-objc',
    'vendor/tree-sitter-fortran'

  ]
)

False

## 导入treesitter库，生成各个语言

In [13]:
from tree_sitter import Language, Parser
CPP_LANGUAGE = Language('build/my-languages.so', 'cpp')
PY_LANGUAGE = Language('build/my-languages.so', 'python')
JAVA_LANGUAGE=Language('build/my-languages.so', 'java')
JS_LANGUAGE=Language('build/my-languages.so', 'javascript')
C_SHARP_LANGUAGE=Language('build/my-languages.so', 'c_sharp')
GO_LANGUAGE=Language('build/my-languages.so', 'go')
PHP_LANGUAGE=Language('build/my-languages.so', 'php')
TYPESCRIPT_LANGUAGE=Language('build/my-languages.so','tsx')
C_LANGUAGE=Language('build/my-languages.so','c')
CSS_LANGUAGE=Language('build/my-languages.so','css')
PROLOG_LANGUAGE=Language('build/my-languages.so','prolog')
R_LANGUAGE=Language('build/my-languages.so','r')
RUST_LANGUAGE=Language('build/my-languages.so','rust')
CUDA_LANGUAGE=Language('build/my-languages.so','cuda')
PERL_LANGUAGE=Language('build/my-languages.so','perl')
SCALA_LANGUAGE=Language('build/my-languages.so','scala')
MATLAB_LANGUAGE=Language('build/my-languages.so','matlab')
RUBY_LANGUAGE=Language('build/my-languages.so','ruby')
PASCAL_LANGUAGE=Language('build/my-languages.so','pascal')
LUA_LANGUAGE=Language('build/my-languages.so','lua')
#SWIFT_LANGUAGE=Language('build/my-languages.so','swift')
#LEAN_LANGUAGE=Language('build/my-languages.so','lean')
DART_LANGUAGE=Language('build/my-languages.so','dart')
KOTLIN_LANGUAGE=Language('build/my-languages.so','kotlin')
HTML_LANGUAGE=Language('build/my-languages.so','html')
LISP_LANGUAGE=Language('build/my-languages.so','elisp')
LATEX_LANGUAGE=Language('build/my-languages.so','latex')
#SHELL_LANGUAGE=Language('build/my-languages.so','powershell')
SQL_LANGUAGE=Language('build/my-languages.so','sqlite')
OBJC_LANGUAGE=Language('build/my-languages.so','objc')
FORTRAN_LANGUAGE=Language('build/my-languages.so','fortran')

## 生成嵌套列表格式的ast
函数process_string(text,left='[[]',right='[]]',sep=','),将lisp风格的括号嵌套字符串转化为python列表。
exp:(module(indicator))=>['module',['indicator']]
函数code2ast(language:string,filepath:string)
exp:
"examples/example_python.py"

In [14]:
import re
def process_string(text,left='[[]',right='[]]',sep=' '):
    pat = r'({}|{}|{})'.format(left, right, sep)
    tokens = re.split(pat, text)
    stack = [[]]
    for x in tokens:
        if not x or re.match(sep, x): 
            continue
        if re.match(left, x):
            stack[-1].append([])
            stack.append(stack[-1][-1])
        elif re.match(right, x):
            stack.pop()
            if not stack:
                return 'error: opening bracket is missing'
                #raise ValueError('error: opening bracket is missing')
        else:
            stack[-1].append(x)
    if len(stack) > 1:
        return 'error: closing bracket is missing'
        raise ValueError('error: closing bracket is missing')
    return stack.pop()

In [15]:

def getast_tree(language,text):
    LANGUAGE=Language('build/my-languages.so', language)
    parser=Parser()
    parser.set_language(LANGUAGE)
    tree=parser.parse(bytes(text,"utf8"))
    return tree
def getast_list(language:str,text:str):
    tree=getast_tree(language,text)
    root_node=tree.root_node
    string=root_node.sexp()
    string=string.replace('(','[')
    string=string.replace(')',']')
    return process_string(string)
def code2ast(language,filepath):
    with open(filepath, "r") as f:
        code = f.read()
    return getast_list(language,code)

### 测试test

In [16]:
'''
print("Cpp")
print(code2ast("cpp","examples/example_cpp.cpp"))
print("\n")
print("Python")
print(code2ast("python","examples/example_python.py"))
print("\n")
print("java")
print(code2ast("java","examples/example_java.java"))
print("\n")
print("JS")
print(code2ast("javascript","examples/example_js.js"))
print("\n")
print("Go")
print(code2ast("go","examples/example_go.go"))
print("\n")
print("php")
print(code2ast("php","examples/example_php.php"))
print("\n")
print("C-sharp")
print(code2ast("c_sharp","examples/example_c_sharp.cs"))
print("\n")
print("TS")
print(code2ast("tsx","examples/example_ts.ts"))
print("\n")
'''
print('C')
print(code2ast("c","examples/example_c.c"))
print('\n')
'''
print('css')
print(code2ast("css","examples/example_css.css"))
print('prolog')
print(code2ast("prolog","examples/example_prolog.pl"))
print('r')
print(code2ast("r","examples/example_r.r"))
print('\n')
print('rust')
print(code2ast("rust","examples/example_rust.rs"))
print('\n')
print('cuda')
print(code2ast("cuda","examples/example_cuda.cu"))
print('\n')
print('perl')
print(code2ast("perl","examples/example_perl.pl"))
print('\n')
print('scala')
print(code2ast("scala","examples/example_scala.scala"))
print('\n')
print('matlab')
print(code2ast("matlab","examples/example_matlab.m"))
print('\n')
print('pascal')
print(code2ast("pascal","examples/example_pascal.pas"))
print('\n')
print('ruby')
print(code2ast("ruby","examples/example_ruby.rb"))
print('\n')
print('lua')
print(code2ast("lua","examples/example_lua.lua"))
print('\n')
print('dart')
print(code2ast("dart","examples/example_dart.dart"))
print('\n')
print('kotlin')
print(code2ast("kotlin","examples/example_kotlin.kt"))
print('\n')
print('html')
print(code2ast("html","examples/example_html.html"))
print('\n')
print('lisp')
print(code2ast("kotlin","examples/example_lisp.lisp"))
print('\n')
print('tex')
print(code2ast("latex","examples/example_tex.tex"))
print('\n')
print('sql')
print(code2ast("sqlite","examples/example_sql.sql"))
print('\n')
print('object-c')
print(code2ast("objc","examples/example_objc.objc"))
print('\n')
print('fortran')
print(code2ast("fortran","examples/example_fortran.f90"))
print('\n')'''



C
[['translation_unit', ['comment'], ['function_definition', 'type:', ['sized_type_specifier', 'type:', ['primitive_type']], 'declarator:', ['function_declarator', 'declarator:', ['identifier'], 'parameters:', ['parameter_list', ['parameter_declaration', 'type:', ['primitive_type'], 'declarator:', ['pointer_declarator', 'declarator:', ['identifier']]], ['parameter_declaration', 'type:', ['primitive_type'], 'declarator:', ['identifier']]]], 'body:', ['compound_statement', ['declaration', 'type:', ['sized_type_specifier', 'type:', ['primitive_type']], 'declarator:', ['init_declarator', 'declarator:', ['identifier'], 'value:', ['call_expression', 'function:', ['identifier'], 'arguments:', ['argument_list', ['identifier'], ['identifier']]]]], ['comment'], ['if_statement', 'condition:', ['parenthesized_expression', ['unary_expression', 'argument:', ['identifier']]], 'consequence:', ['return_statement', ['identifier']]], ['comment'], ['expression_statement', ['call_expression', 'function:', 

'\nprint(\'css\')\nprint(code2ast("css","examples/example_css.css"))\nprint(\'prolog\')\nprint(code2ast("prolog","examples/example_prolog.pl"))\nprint(\'r\')\nprint(code2ast("r","examples/example_r.r"))\nprint(\'\n\')\nprint(\'rust\')\nprint(code2ast("rust","examples/example_rust.rs"))\nprint(\'\n\')\nprint(\'cuda\')\nprint(code2ast("cuda","examples/example_cuda.cu"))\nprint(\'\n\')\nprint(\'perl\')\nprint(code2ast("perl","examples/example_perl.pl"))\nprint(\'\n\')\nprint(\'scala\')\nprint(code2ast("scala","examples/example_scala.scala"))\nprint(\'\n\')\nprint(\'matlab\')\nprint(code2ast("matlab","examples/example_matlab.m"))\nprint(\'\n\')\nprint(\'pascal\')\nprint(code2ast("pascal","examples/example_pascal.pas"))\nprint(\'\n\')\nprint(\'ruby\')\nprint(code2ast("ruby","examples/example_ruby.rb"))\nprint(\'\n\')\nprint(\'lua\')\nprint(code2ast("lua","examples/example_lua.lua"))\nprint(\'\n\')\nprint(\'dart\')\nprint(code2ast("dart","examples/example_dart.dart"))\nprint(\'\n\')\nprint(\